In [2]:
import os
import json
from pathlib import Path
import pandas as pd
from datasets import Dataset,load_dataset

In [3]:
root_path = '/mnt/users/LLMa/ChatGLM-6B'

output_folder= 'lung_conclusion-chatglm-6b-pt1-128-1e-3'

gt_json = os.path.join(root_path,"data/LUNG/val_conclusion_500.json")
pred_json = os.path.join(root_path,"output",output_folder,  "generated_predictions.txt")
csv_file = os.path.join(root_path,"output",output_folder,"compose_lung_0424.xlsx")

In [8]:
gt_data = load_dataset('json',data_files=gt_json,use_auth_token=True)
pred_data = load_dataset('json',data_files=pred_json,use_auth_token=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /mnt/users/.cache/huggingface/datasets/json/default-4ee0f01b55834b32/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/mnt/users/.cache/huggingface/datasets/json/default-691180fe522198de/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
inputs = gt_data['train']['Input']
labels  = gt_data['train']['Output']
preds = pred_data['train']['predict']
compose_data = Dataset.from_dict({'Report':inputs,
                                  'Doc_Conclusion':labels,
                                  'AI_Conclusion':preds})
df = compose_data.to_pandas()

writer = pd.ExcelWriter(csv_file, engine='xlsxwriter')

df.to_excel(writer,index=False,sheet_name='Sheet1')
worksheet = writer.sheets['Sheet1']
worksheet.set_column("A:C", 72)
worksheet.set_default_row(40)
workbook  = writer.book  

format = workbook.add_format({
    'text_wrap': True,
    'valign': 'top',})
for row_num,row in df.iterrows():
    for col_num, value in enumerate(df.columns.values):  
        worksheet.write(row_num+1, col_num, row[value], format)
writer.close()